<a href="https://colab.research.google.com/github/DmitryKutsev/cross-document_coreference/blob/main/text_analysis_coref.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download ru_core_news_sm
!python -m spacy download ru_core_news_md

2021-05-05 20:31:20.699537: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
     |████████████████████████████████| 17.9 MB 1.3 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
2021-05-05 20:31:30.294119: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
     |████████████████████████████████| 43.9 MB 18 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_md')


In [48]:
pip install pymorphy2[fast]

In [49]:
pip install Gensim

In [50]:
import spacy
import gensim
from gensim.models import Word2Vec 
from nltk.tokenize import RegexpTokenizer
from nltk import  sent_tokenize
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [51]:
nlp = spacy.load('ru_core_news_md')

In [52]:
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/neuroleptic_ru.txt

--2021-05-05 20:31:53--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/neuroleptic_ru.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3638613 (3.5M) [text/plain]
Saving to: ‘neuroleptic_ru.txt.1’

neuroleptic_ru.txt. 100%[===================>]   3.47M  13.8MB/s    in 0.3s    

2021-05-05 20:31:54 (13.8 MB/s) - ‘neuroleptic_ru.txt.1’ saved [3638613/3638613]



In [53]:
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/rakpobedim_ru.txt

--2021-05-05 20:31:54--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/rakpobedim_ru.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12963713 (12M) [text/plain]
Saving to: ‘rakpobedim_ru.txt.1’

rakpobedim_ru.txt.1 100%[===================>]  12.36M  24.0MB/s    in 0.5s    

2021-05-05 20:31:55 (24.0 MB/s) - ‘rakpobedim_ru.txt.1’ saved [12963713/12963713]



In [54]:
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/hv-info_gepatit.txt

--2021-05-05 20:31:55--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/hv-info_gepatit.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6298252 (6.0M) [text/plain]
Saving to: ‘hv-info_gepatit.txt.1’

hv-info_gepatit.txt 100%[===================>]   6.01M  16.4MB/s    in 0.4s    

2021-05-05 20:31:55 (16.4 MB/s) - ‘hv-info_gepatit.txt.1’ saved [6298252/6298252]



In [55]:
!wget https://rusvectores.org/static/models/news_upos_cbow_300_2_2017.bin.gz

--2021-05-05 20:31:56--  https://rusvectores.org/static/models/news_upos_cbow_300_2_2017.bin.gz
Resolving rusvectores.org (rusvectores.org)... 116.203.104.23
Connecting to rusvectores.org (rusvectores.org)|116.203.104.23|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 140310878 (134M) [application/x-gzip]
Saving to: ‘news_upos_cbow_300_2_2017.bin.gz.2’

news_upos_cbow_300_ 100%[===================>] 133.81M  21.0MB/s    in 6.9s    

2021-05-05 20:32:03 (19.3 MB/s) - ‘news_upos_cbow_300_2_2017.bin.gz.2’ saved [140310878/140310878]



In [56]:
handler = open('neuroleptic_ru.txt', 'r', encoding='utf-8')
my_txt = handler.read()
handler.close()


In [57]:
handler = open('hv-info_gepatit.txt', 'r', encoding='utf-8')
my_txt += handler.read()
handler.close()

In [58]:
handler = open('rakpobedim_ru.txt', 'r', encoding='utf-8')
my_txt += handler.read()
handler.close()

In [59]:
len(my_txt)

13016908

In [60]:
tfidf = TfidfVectorizer(max_features=3000, ngram_range=(1,3))

In [61]:
!wget https://rusvectores.org/static/models/news_upos_cbow_300_2_2017.bin.gz

--2021-05-05 20:32:04--  https://rusvectores.org/static/models/news_upos_cbow_300_2_2017.bin.gz
Resolving rusvectores.org (rusvectores.org)... 116.203.104.23
Connecting to rusvectores.org (rusvectores.org)|116.203.104.23|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 140310878 (134M) [application/x-gzip]
Saving to: ‘news_upos_cbow_300_2_2017.bin.gz.3’

news_upos_cbow_300_ 100%[===================>] 133.81M  23.3MB/s    in 6.4s    

2021-05-05 20:32:10 (20.9 MB/s) - ‘news_upos_cbow_300_2_2017.bin.gz.3’ saved [140310878/140310878]



In [62]:
posts = str_list = my_txt.split('----')

In [63]:
tokenizer = RegexpTokenizer(r'\w+')
sentences_tokenized = [w.lower() for w in posts]

vectorized_txt = tfidf.fit_transform(sentences_tokenized)
sentences_tokenized = [tokenizer.tokenize(i) for i in sentences_tokenized]

In [64]:
from gensim.models import FastText
from google.colab import files

In [65]:
ft = FastText(sentences_tokenized, size=100, window=5, min_count=5, workers=4,sg=1)

In [66]:
ft.most_similar('алимемазин')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('левомепромазин', 0.8881936073303223),
 ('хлорпромазин', 0.863217830657959),
 ('трифлуоперазин', 0.8540201187133789),
 ('тиоридазин', 0.8524168729782104),
 ('флупентиксол', 0.8407307267189026),
 ('саркозин', 0.839452862739563),
 ('тизерцин', 0.8383066654205322),
 ('зуклопентиксол', 0.8367112874984741),
 ('палиперидон', 0.8348816633224487),
 ('флупентиксола', 0.8264579176902771)]

In [67]:
ft.wv.__getitem__('алимемазин')

array([-0.19141215, -0.17774145, -0.22505116,  0.3930469 ,  0.2722114 ,
       -0.08688641,  0.0538663 , -0.00939174, -0.09319028,  0.1759918 ,
       -0.23820367,  0.34295017,  0.07379159, -0.06503055, -0.43044055,
        0.07127316,  0.45762634, -0.20185265,  0.10893501, -0.0715092 ,
        0.17989086,  0.22505763, -0.07567516,  0.37397075,  0.06719073,
       -0.19658335,  0.05161051, -0.09121012,  0.55530864, -0.10352173,
       -0.3504836 ,  0.30780098, -0.08295261, -0.14436   ,  0.08618001,
        0.10153401,  0.03278841, -0.0530084 , -0.14843382,  0.28717676,
        0.3605114 ,  0.11710894, -0.6245839 , -0.03557574,  0.0619835 ,
        0.04350219,  0.2219474 ,  0.04994988, -0.20186073,  0.01828795,
       -0.43029007, -0.23079708, -0.35106277, -0.08056272, -0.21141922,
       -0.13126785,  0.21252218, -0.06816931,  0.03809527, -0.06911878,
        0.06998801,  0.28292462, -0.4726258 ,  0.01234577,  0.09673467,
        0.23618829, -0.24593511, -0.07939471,  0.21396916,  0.05

In [68]:
cosine_similarity(np.atleast_2d(model.wv.get_vector('алимемазин')), \
                  np.atleast_2d(model.wv.get_vector('азалептин')))[0][0]

NameError: ignored

In [ ]:
files.download('small_fasttext')

In [ ]:
w2v = gensim.models.KeyedVectors.load_word2vec_format('news_upos_cbow_300_2_2017.bin.gz', binary=True)

In [ ]:
model = Word2Vec(size=300, min_count=1)
model.build_vocab(sentences_tokenized)
total_examples = model.corpus_count
model.build_vocab([list(w2v.vocab.keys())], update=True)
model.intersect_word2vec_format("news_upos_cbow_300_2_2017.bin.gz", binary=True)
model.train(sentences_tokenized, total_examples=total_examples, epochs=model.iter)

In [ ]:
# int(model['препарат'][-1])
model.wv.__getitem__('препарат')[-1]

In [ ]:
model.most_similar('препарат')

In [ ]:
# model['алимемазин']
model.wv.most_similar('алимемазин')

In [ ]:
cosine_similarity(np.atleast_2d(ft.wv.get_vector('алимемазин')), \
                  np.atleast_2d(ft.wv.get_vector('азалептин')))[0][0]

In [ ]:
cosine_similarity(np.atleast_2d(ft.wv.get_vector('алимемазин')), \
                  np.atleast_2d(ft.wv.get_vector('стул')))[0][0]

In [ ]:
cosine_similarity(np.atleast_2d(ft.wv.get_vector('дак')), \
                  np.atleast_2d(ft.wv.get_vector('соф')))[0][0]

In [ ]:
cosine_similarity(np.atleast_2d(ft.wv.get_vector('дак')), \
                  np.atleast_2d(ft.wv.get_vector('софосбувир')))[0][0]

In [ ]:
cosine_similarity(np.atleast_2d(ft.wv.get_vector('соф')), \
                  np.atleast_2d(ft.wv.get_vector('софосбувир')))[0][0]

In [70]:
# Модель есть, теперь надо набрать корпус из трех форумов примерно поровну, и чтобы не помереть при разметке.

СОВСЕМ НЕМНОЖКО, для начала!!


In [143]:
short_txt = " "
handler = open('neuroleptic_ru.txt', 'r', encoding='utf-8')
short_txt = handler.read()[:20000]
handler.close()

handler = open('rakpobedim_ru.txt', 'r', encoding='utf-8')
short_txt += handler.read()[:20000]
handler.close()

handler = open('hv-info_gepatit.txt', 'r', encoding='utf-8')
short_txt += handler.read()[:20000]
handler.close()

In [144]:
len(short_txt)

60000

In [145]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [146]:
short_posts = str_list = short_txt.split('----')
short_posts[10]

'\nНа рынок снова вернулся алимемазин (раньше терален назывался). Хотелось бы узнать Ваше мнение. Кто принимал в своей практике, что хорошего, что плохого\n\n____________________________________________\nСообщение отредактировано.\n\nДобавлена ссылка на ролик в You Tube.\n\nСообщение отредактировал Гилев: 12 Апрель 2020 - 07:55\nДобавлена ссылка на ролик в You Tube\n'

In [147]:
len(short_posts)


194

In [149]:
# Словарь предложение:номер поста
tokenizer = RegexpTokenizer(r'\w+')
sent_dict = {}
for num, post in enumerate(short_posts):
  for sent in nltk.sent_tokenize(post):
    sent = ' '.join(tokenizer.tokenize(sent))
    sent = sent.replace('_', '') 
    # sent = nltk.regexp_tokenize(sent, '[а-яА-яa-zA-Z.!? ]')
    if sent:
      sent_dict[sent] = num 

In [150]:
len(sent_dict)

537

In [151]:
# только для удобства, если не будет влезать в память - использую ключи словаря
sent_list = list(sent_dict.keys())
sent_list[:9]

['На рынок снова вернулся алимемазин раньше терален назывался',
 'Хотелось бы узнать Ваше мнение',
 'Кто принимал в своей практике что хорошего что плохого  Сообщение отредактировано',
 'Добавлена ссылка на ролик в You Tube',
 'Сообщение отредактировал Гилев 12 Апрель 2020 07 55 Добавлена ссылка на ролик в You Tube',
 'Орто сказал а 27 11 2008 19 59 На рынок снова вернулся алимемазин раньше терален назывался',
 'Кто принимал в своей практике что хорошего что плохого Это тот редкий случай когда антипсихотик оставил самые приятные впечатления к сожалению про моно терапию рассказать не могу т к принимал его совместно с антидепрессантами Орто а где его можно приобрести не подскажете',
 'В Волгограде есть почти во всех аптеках стоит 80 руб',
 'Мы его применяемм в основном как вегетостабилизатор']

In [152]:
sent_dict[sent_list[0]]

10

In [153]:

# sentences_tokenized = [w.lower() for w in short_posts]
# sentences_tokenized = [tokenizer.tokenize(i) for i in sentences_tokenized]
# sentences_tokenized[0]

In [154]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [155]:
sample = 'он'
print('Одуш', morph.parse(sample)[0].tag.animacy)
print('Падеж', morph.parse(sample)[0].tag.case )
print('Пол', morph.parse(sample)[0].tag.gender)
print('Число', morph.parse(sample)[0].tag.number)
print('Лицо', morph.parse(sample)[0].tag.person)
print('Часть речи', morph.parse(sample)[0].tag.POS)
print('Время', morph.parse(sample)[0].tag.tense)
print('Залог', morph.parse(sample)[0].tag.voice)



Одуш None
Падеж nomn
Пол masc
Число sing
Лицо 3per
Часть речи NPRO
Время None
Залог None


In [156]:
aa = morph.parse(sample)[0]
print(aa.tag)

NPRO,masc,3per,Anph sing,nomn


In [157]:
sent_list[61:89]

['Похоже он у меня работает как аугментация к АД',
 'Сегодня повысил ципралекс до 10мг сначала размазало а сейчас вечером прям норм стало даже повышенное настроение и голова заработала',
 'Кстати какой ад на либидыч меньше воздействует ципра или сертралин',
 'возможно сертралин',
 'Из побочек добавилась бессонница',
 'Я для сна при приеме карипразина пил еще 1 4 тизирцина для сна впринципе работало но неделе через 2 появилась бессонница мог спать по 3 часа Или не мог долго уснуть',
 'Из побочек еще тревога сильная была',
 'Он вообще препарат стимулирующий',
 'Обсуждаем препарат Сероквель Кветиапин',
 'Повышает ли сероквель уровень пролактина в крови',
 'Есть ли после него акатизия тремор В основном какие побочные встречаются при прееме Сероквеля',
 'На уровень пролактина Сероквель практически не влияет',
 'Экстрапирамидные осложнения также бывают редко',
 'Этот препарат отличается хорошей переносимостью',
 'Лечит ли Сероквель негативную симптоматику',
 'Нужно отметить что к сожалению С

In [158]:
sent_list[66]

'Я для сна при приеме карипразина пил еще 1 4 тизирцина для сна впринципе работало но неделе через 2 появилась бессонница мог спать по 3 часа Или не мог долго уснуть'

In [159]:
doc = nlp(sent_list[66])
for token in doc:
    print(token.text, token.dep_, token.head.text, token.head.pos_, 
            [child for child in token.children], morph.parse(token.text)[0].tag,
          [child for child in token.head.children])


Я nsubj пил VERB [] NPRO,1per sing,nomn [Я, сна, приеме, тизирцина, впринципе, работало, появилась, мог, мог]
для case сна NOUN [] PREP [для]
сна obl пил VERB [для] NOUN,inan,masc sing,gent [Я, сна, приеме, тизирцина, впринципе, работало, появилась, мог, мог]
при case приеме NOUN [] PREP [при, карипразина]
приеме obl пил VERB [при, карипразина] NOUN,inan,masc sing,loct [Я, сна, приеме, тизирцина, впринципе, работало, появилась, мог, мог]
карипразина nmod приеме NOUN [] NOUN,anim,masc,Sgtm,Surn sing,gent [при, карипразина]
пил ROOT пил VERB [Я, сна, приеме, тизирцина, впринципе, работало, появилась, мог, мог] VERB,impf,tran masc,sing,past,indc [Я, сна, приеме, тизирцина, впринципе, работало, появилась, мог, мог]
еще advmod 1 NUM [] ADVB [еще]
1 nummod 4 NUM [еще] NUMB,intg [1]
4 nummod:gov тизирцина NOUN [1] NUMB,intg [4, сна]
тизирцина obj пил VERB [4, сна] NOUN,anim,masc,Sgtm,Surn sing,gent [Я, сна, приеме, тизирцина, впринципе, работало, появилась, мог, мог]
для case сна NOUN [] PREP

In [160]:
doc = nlp(sent_list[61])
for token in doc:
    print(token.text, token.dep_, token.head.text, token.head.pos_,
            [child for child in token.children], morph.parse(token.text)[0].tag,
          [child for child in token.head.children],)


Похоже advmod работает VERB [] ADVB [Похоже, он, меня, аугментация, АД]
он nsubj работает VERB [] NPRO,masc,3per,Anph sing,nomn [Похоже, он, меня, аугментация, АД]
у case меня PRON [] PREP [у]
меня obl работает VERB [у] NPRO,1per sing,accs [Похоже, он, меня, аугментация, АД]
работает ROOT работает VERB [Похоже, он, меня, аугментация, АД] VERB,impf,intr sing,3per,pres,indc [Похоже, он, меня, аугментация, АД]
как case аугментация NOUN [] CONJ [как]
аугментация obl работает VERB [как] NOUN,inan,femn sing,nomn [Похоже, он, меня, аугментация, АД]
к case АД PROPN [] PREP [к]
АД obl работает VERB [к] NOUN,inan,masc,Sgtm sing,nomn [Похоже, он, меня, аугментация, АД]


In [162]:
sample = 'он'
print('Одуш', morph.parse(sample)[0].tag.animacy)
print('Падеж', morph.parse(sample)[0].tag.case )
print('Пол', morph.parse(sample)[0].tag.gender)
print('Число', morph.parse(sample)[0].tag.number)
print('Лицо', morph.parse(sample)[0].tag.person)
print('Часть речи', morph.parse(sample)[0].tag.POS)
print('Время', morph.parse(sample)[0].tag.tense)
print('Залог', morph.parse(sample)[0].tag.voice)


Одуш None
Падеж nomn
Пол masc
Число sing
Лицо 3per
Часть речи NPRO
Время None
Залог None


***Разметка для кореференции***

In [163]:
coref_df = pd.DataFrame(columns=['TOKEN', 'TOKEN_VECT', 'POST_NUM', 'ANIMACY', 'CASE', 'GENDER', 'PERSON', 'POS', 'DEPENDENCY', 'HEAD', 'HEAD_VECT' 'HEAD_POS',
                                 'HEAD_ANIMACY', 'HEAD_CASE', 'HEAD_GENDER',  'HEAD_POS', 'SENT_VECTOR', 'HEAD_CHILDS_VECT', 'NER',
                                 'COREFERENCE_CLUSTER', ])

In [164]:
sent_vector = 0
for sent in sent_list:
  sent_vector = 0
  post_num = sent_dict.get(sent)
  # sent_vector = sum([model.wv.__getitem__(tok.lower())[-1] for tok in sent.split()])
  
  for i in sent.split():
    try:     
      sent_vector += ft.wv.__getitem__(i.lower())[-1]
    except KeyError:
      print(i)
  sent_vector = sent_vector/len(sent.split())
  # if sent_vector == '-' or None or 0:
  #   print(sent)
  my_sent = nlp(sent)
  for token in my_sent:
    morphy = morph.parse(token.text)[0]

    app_dict = {}
    app_dict['TOKEN'] = token.text or '-'
    try:
      app_dict['TOKEN_VECT'] = ft.wv.__getitem__(token.text.lower())[-1] if token.text else '-'
    except KeyError:
      print(token.text.lower())
    app_dict['POST_NUM'] = post_num or '-'
    app_dict['ANIMACY'] = morphy.tag.animacy or '-'
    app_dict['CASE'] = morphy.tag.case or '-'
    app_dict['GENDER'] = morphy.tag.gender or '-'
    app_dict['PERSON'] = morphy.tag.person or '-'
    app_dict['POS'] = morphy.tag.POS or '-' 
    app_dict['DEPENDENCY'] = token.dep_ or '-'

    app_dict['HEAD'] = token.head.text or '-'
    try:
      app_dict['HEAD_VECT'] = ft.wv.__getitem__(token.head.text)[-1] if token.head.text else '-'
    except KeyError:
      print(token.head.text)
    app_dict['HEAD_POS'] = token.head.pos_ or '-'
    morphy = morph.parse(token.head.text)[0] or '-'
    app_dict['HEAD_ANIMACY'] = morphy.tag.animacy or '-'
    app_dict['HEAD_CASE'] = morphy.tag.case or '-'
    app_dict['HEAD_GENDER'] = morphy.tag.gender or '-'
    # app_dict['HEAD_PERSON'] = morphy.tag.person or '-'
    app_dict['HEAD_POS'] = morphy.tag.POS or '-'
    app_dict['SENT_VECTOR'] = sent_vector or '-'
    try:
      if token.head.children:
        app_dict['HEAD_CHILDS_VECT'] = 0
        for child in token.head.children:
          app_dict['HEAD_CHILDS_VECT'] += ft[child.text.lower()][-1]
        if list(token.head.children):
          app_dict['HEAD_CHILDS_VECT'] = app_dict['HEAD_CHILDS_VECT']/len(list(token.head.children))
    except KeyError:
      print(child.text.lower())

    app_dict['NER'] = 0
    app_dict['COREFERENCE_CLUSTER'] = 0
    coref_df = coref_df.append(app_dict, ignore_index=True)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


 
 
 
ПП
ПП
D3
d3
d3
D3
Он
АД
D2
Да
По
По
NO
NO
И
УЗИ
УЗИ
Ой
В
В
РПЖ
РПЖ
И
РПЖ
Из
РПЖ
РПЖ
ЛП
C
3Mb
3mb
PDF
3mb
PDF
3mb
PDF
3mb
PDF
3mb
PDF
3mb
PDF
3mb
PDF
3mb
PDF
3mb
PDF
3mb
PDF
3mb
PDF
3mb
PDF
3mb
PDF
3mb
ППД
Да
Да
БЛ
БЛ
БЛ
Но
Но
По
Р
УВО
PR
PR
Да
Да
PR
PR
ППД
ППД
ОЗТ
ППД
ОЗТ
RATING
УВО
УВО
УВО
УВО
КИ
URL
URL
URL
URL
URL
URL
URL
ВСЕХ
А
Т
Т
Ну
Ну
Ок
Ок
Ок
EASL
1a
А1
1a
1a
а1
а1
а1
а1
Вы


In [165]:
coref_df

,TOKEN,TOKEN_VECT,POST_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_VECTHEAD_POS,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,SENT_VECTOR,HEAD_CHILDS_VECT,NER,COREFERENCE_CLUSTER,HEAD_VECT
0,На,-0.046335,10,-,-,-,-,PREP,case,рынок,NaN,inan,accs,masc,NOUN,0.094495,-0.046335,0,0,0.150048
1,рынок,0.150048,10,inan,accs,masc,-,NOUN,obl,вернулся,NaN,-,-,masc,VERB,0.094495,0.056298,0,0,0.284159
2,снова,-0.071036,10,-,-,-,-,ADVB,advmod,вернулся,NaN,-,-,masc,VERB,0.094495,0.056298,0,0,0.284159
3,вернулся,0.284159,10,-,-,masc,-,VERB,ROOT,вернулся,NaN,-,-,masc,VERB,0.094495,0.056298,0,0,0.284159
4,алимемазин,-0.023104,10,inan,nomn,masc,-,NOUN,nsubj,вернулся,NaN,-,-,masc,VERB,0.094495,0.056298,0,0,0.284159
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6674,3,-0.009287,193,-,-,-,-,-,ROOT,3,NaN,-,-,-,-,-0.124611,0.000000,0,0,-0.009287
6675,Пациенты,0.141368,193,anim,nomn,masc,-,NOUN,ROOT,Пациенты,NaN,anim,nomn,masc,NOUN,-0.219679,-0.072722,0,0,0.198803
6676,принимавшие,0.195333,193,-,nomn,-,-,PRTF,acl,Пациенты,NaN,anim,nomn,masc,NOUN,-0.219679,-0.072722,0,0,0.198803
6677,софосбувир,-0.874638,193,anim,gent,femn,-,NOUN,obj,принимавшие,NaN,-,nomn,-,PRTF,-0.219679,-0.874638,0,0,0.195333


In [166]:
coref_df.to_csv('toy_coref_fasttext.csv', sep=' ', index=False)

In [167]:
from google.colab import files

In [168]:
files.download('toy_coref_fasttext.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>